In [1]:
# request site text, search for certain tags and write them, then put them in doc. To run again, need to delete file.html and docx file
from bs4 import BeautifulSoup as bs
import requests
import pypandoc
import pandas as pd

#line types in html file that will be checked.
line_types = ["h1","h2","h3","h4","h5","p","ol","ul"]  # there is an issue getting the <li> out of nested lists without repeting.

# URl to get the html file from
URL = "https://houseofyumm.com/chile-colorado/"

# get webpage and make a html.parser of that page using BeautifulSoup
page = requests.get(URL)
soup = bs(page.content, "html.parser")

# create a file in the same folder as this program and start writing.
with open('file.html', 'w') as file:
    start = True
    for data in soup.find_all(line_types):
        file.write(str(data))

# convert file.html inot a word docx
output = pypandoc.convert_file('file.html', 'docx', outputfile="file1.docx")
assert output == ""


OSError: [Errno 30] Read-only file system: 'file.html'

In [7]:
# need to install these packeages using pip or conda in console before running.
from bs4 import BeautifulSoup as bs
import requests
import pypandoc
import pandas as pd

# This is a list of the line types in html file that will be checked.
# TODO There is an issue where ul and ol elements contain li elements.
# Also, li elements could contain sublists. Both of these issues
# causes problems with writing an elements multiple times in the html file.
# Examle: We could write an ul element, and a li lement contained in it. Or a li element
# could contain a ul element with further li elements.
# One way to solve this would be to write a program that takes an ol or ul and
# writes it, does not write all ol and ul contained in it. Then simple do
# not search for li elements, since they are contained in ol and ul (right?)

line_types = ["h1","h2","h3","h4","h5","p","ol","ul","li"]

# we read an excel sheet in the same folder as this file and go down the first column, trying to upen URLs there
df = pd.read_excel('Cherri Shelton.xlsx')

# We will write a html file holding all the content we scrape from webpages, then save that file to a word doc
with open('file.html', 'w') as file:

    # we will do the following for each url in the excel column
    for URL in df.iloc[:,0]:

        # try to open the url. Some entries in the doc may not be URLs, or the links may be broken.
        # TODO: Perhaps we should include a case if a URL is broken, as opposed to just passing through.
        try:
            # try to open the URL and get the content ready to parse using BeautifulSoup

            page = requests.get(URL)
            soup = bs(page.content, "html.parser")
            
            # create a heading in HTML that will be easily seen when we convert to a word doc
            file.write("<p>#############################################</p>")
            file.write(URL)
            file.write("<p>#############################################</p>")

            # the below gets any html element that has a label in "line_types" and writes them to file.html.
            # It gets the elemnts in the order they appear in the original html file. There is also an attempt
            # to avoid unwanted information by checking for a condition until it starts writing content.
            # The condition looks for a series of headings (<h1>, <h2>, etc.) followed by a <p> element.
            # when this is satisfied, it writed the series of headings, the <p> element, and all later elements.
            # TODO this logic is not very good. THe next box has an attempt as better logic.

            # start = 0 at the beginning
            mode = "searching for heading"
            for data in soup.find_all(line_types):

                # if moe == "searching for heading", parser looks for a heading. When a heading is found, it starts storing lines
                if mode == "searching for heading":
                    if data.name == "h1" or data.name == "h2" or data.name == "h3" or data.name == "h4" or data.name == "h5":
                        string_storage = str(data)
                        mode = "looking for paragraph"
                        
                # If mode == "searching for paragraph", the parser has found a heading, and is searching for paragraphs (<p>). If it finds another heading,
                # it will store that and continue looking for paragraphs. If it finds something that is a paragraph, it will write everything it has
                # stored and turn to full record mode. If it finds soemthing else, it will turn to "searching for heading" mode.
                elif mode == "looking for paragraph":
                    if data.name == "h1" or data.name == "h2" or data.name == "h3" or data.name == "h4" or data.name == "h5":
                        string_storage += str(data)
                    elif data.name == "p":
                        file.write(string_storage + str(data))
                        mode = "full record"
                    else:
                        mode = "searching for heading"
                else:
                    file.write(str(data))

            # After the whole file has been parsed, we write a newline to separate webpages in the doc file.
            file.write('<p><br></p>')
        except:
            pass

# after we have written the html file with all the websites, we convert the html file to a docx file.
output = pypandoc.convert_file('file.html', 'docx', outputfile="file1.docx")
assert output == ""

In [8]:
# below is another file parser with some more complex logic.
from bs4 import BeautifulSoup as bs
import requests
import pypandoc
import pandas as pd

# the function takes a data entry and gives the number of words in that entry
def get_entry_length(line):
    return len(line.get_text().split())


line_types = ["h1","h2","h3","h4","h5","p","li"]
# we are looking for paragraph or list elements that are a certain length. This may be an indicator of
# significant content that should be recorded.
length_threshold = 10
df = pd.read_excel('Cherri Shelton.xlsx')
with open('file.html', 'w') as file:
    for URL in df.iloc[:,0]:
        try:
            page = requests.get(URL)
            soup = bs(page.content, "html.parser")
            file.write("<p>#############################################</p>")
            file.write(URL)
            file.write("<p>#############################################</p>")
            string_storage = ''
            recording = False
            recording_body = False
            commit = False
            for data in soup.find_all(line_types):
                # The logic looks for a section of text of a cerain form. It looks for a series of headers, followed by body content line with lots of words.
                # If it finds this, it will record it and start searching again.

                # The logic checks for a heading first. When it finds a heading, it starts recording lines and checks for more headings, paragraphs, or lists.
                # If it finds a paragraph or list element with a length of text that is long (>=10), it will write the saved chunk of headings and paragraphs,
                # and continue to write paragraphs and list elelements until if finds another header. When it finds another header, it will reset.

                # TODO: The search does not look for ol or ul items, so the bullet and numbered format is absent in the word doc. Also, the 
                # get_entry_length function has issues.

                # TODO There is also the issue outlined in cell 2 with searching for just ol and ul items.
                
                # TODO If given a ul/ol item, get_entry_length adds up all text in all contained li elements and returns the sum.
                # It sould return 0. If given a li element containing a ul or ol, get_entry_length will add up all the
                # words in the ul/ol item.
                if not recording:
                    if data.name == "h1" or data.name == "h2" or data.name == "h3" or data.name == "h4" or data.name == "h5":
                        string_storage = str(data)
                        recording = True
                elif not recording_body:
                    if data.name == "h1" or data.name == "h2" or data.name == "h3" or data.name == "h4" or data.name == "h5":
                        string_storage += str(data)
                    elif data.name == "p" or data.name == "li":
                        string_storage += str(data)
                        recording_body = True
                        if get_entry_length(data)>=length_threshold:
                            commit = True
                elif not commit:
                    if data.name == "h1" or data.name == "h2" or data.name == "h3" or data.name == "h4" or data.name == "h5":
                        string_storage = str(data)
                        recording_body = False
                    else:
                        string_storage += str(data)
                        if get_entry_length(data)>=length_threshold:
                            commit = True
                else:
                    # Here is where we write to file. We have recorded a heading followed by body content of a certain length in string_storage,
                    # and have come to another heading. We write the chunk we recorded and begin looking for a body of a certain length.
                    if data.name == "h1" or data.name == "h2" or data.name == "h3" or data.name == "h4" or data.name == "h5":
                        file.write(string_storage)
                        string_storage = str(data)
                        recording_body = False
                        commit = False
                    else:
                        string_storage += str(data)
            
            # If we ended the file and want to commit waht is recorded, we do it here.
            if commit:
                file.write(string_storage)
            file.write('<p><br></p>')
        except:
            pass

output = pypandoc.convert_file('file.html', 'docx', outputfile="file1.docx")
assert output == ""

3
